In [1]:
import sys
sys.path.append("../")

import pandas as pd
import plotly.graph_objects as go
from dateutil import parser
import datetime as dt

from api.oanda_api import OandaApi
from infrastructure.instrument_collection import instrumentCollection as ic
from db.db import DataDB

In [2]:
database = DataDB()

In [3]:
ic.LoadInstrumentsDB()

In [4]:
pairs = []
for pair, val in ic.instruments_dict.items():
    if val.ins_type == "CURRENCY" and "GBP" in pair:
        pairs.append(pair)

In [5]:
pairs

['EUR_GBP',
 'GBP_NZD',
 'GBP_PLN',
 'GBP_CHF',
 'GBP_CAD',
 'GBP_HKD',
 'GBP_ZAR',
 'GBP_JPY',
 'GBP_AUD',
 'GBP_SGD',
 'GBP_USD']

In [6]:
data = []
api = OandaApi()
for p in pairs:
    print(p)
    d_temp = api.get_candles_df(pair_name=p, granularity="D", count=400)
    d_temp['gain'] = ((d_temp['mid-c'] - d_temp['mid-o']) / d_temp['mid-c']) * 100
    d_temp['pair'] = p
    if '_GBP' in p:
        d_temp['gain'] = d_temp['gain'] * -1
    data.append(d_temp)
candles_df = pd.concat(data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

EUR_GBP
GBP_NZD
GBP_PLN
GBP_CHF
GBP_CAD
GBP_HKD
GBP_ZAR
GBP_JPY
GBP_AUD
GBP_SGD
GBP_USD


In [7]:
candles_df.tail()

,volume,time,mid-o,mid-h,mid-l,mid-c,bid-o,bid-h,bid-l,bid-c,ask-o,ask-h,ask-l,ask-c,gain,pair
4395,95703,2024-01-21,1.26885,1.27330,1.26868,1.27082,1.26768,1.27320,1.26768,1.27072,1.27002,1.27340,1.26876,1.27091,0.155018,GBP_USD
4396,103914,2024-01-22,1.27114,1.27473,1.26492,1.26873,1.27026,1.27464,1.26483,1.26864,1.27203,1.27482,1.26500,1.26882,-0.189954,GBP_USD
4397,121144,2024-01-23,1.26800,1.27748,1.26800,1.27242,1.26707,1.27740,1.26707,1.27234,1.26893,1.27757,1.26866,1.27251,0.347370,GBP_USD
4398,123652,2024-01-24,1.27234,1.27426,1.26816,1.27087,1.27185,1.27419,1.26808,1.27077,1.27282,1.27435,1.26824,1.27097,-0.115669,GBP_USD
4399,98389,2024-01-25,1.27076,1.27586,1.26756,1.27022,1.26998,1.27576,1.26746,1.27000,1.27153,1.27595,1.26764,1.27043,-0.042512,GBP_USD


In [8]:
calendar_data = database.query_all(DataDB.CALENDAR_COLL)

In [9]:
calendar_data_df = pd.DataFrame.from_dict(calendar_data)

In [10]:
calendar_data_df.head()

,date,country,category,event,symbol,actual,previous,forecast,data
0,2024-01-23,australia,business confidence,nab business confidence,NABSCONF,,-9,-7,NaN
1,2024-01-24,canada,interest rate,boc monetary policy report,CCLR,,,,NaN
2,2024-01-25,germany,business confidence,ifo business climate,GRIFPBUS,,86.4,87.1,NaN
3,2024-01-26,united states,core pce price index mom,core pce price index mom,USACPPIM,,0.1%,0.2%,NaN
4,2024-01-30,italy,gdp annual growth rate,gdp growth rate yoy adv,ITPIRLYS,,0.1%,-0.3%,NaN


In [11]:
calendar_data_df.date = calendar_data_df.date.dt.date

In [12]:
calendar_data_df

,date,country,category,event,symbol,actual,previous,forecast,data
0,2024-01-23,australia,business confidence,nab business confidence,NABSCONF,,-9,-7,NaN
1,2024-01-24,canada,interest rate,boc monetary policy report,CCLR,,,,NaN
2,2024-01-25,germany,business confidence,ifo business climate,GRIFPBUS,,86.4,87.1,NaN
3,2024-01-26,united states,core pce price index mom,core pce price index mom,USACPPIM,,0.1%,0.2%,NaN
4,2024-01-30,italy,gdp annual growth rate,gdp growth rate yoy adv,ITPIRLYS,,0.1%,-0.3%,NaN
...,...,...,...,...,...,...,...,...,...
13721,2024-02-02,united states,non farm payrolls,non farm payrolls,NFP TCH,,216K,175.0K,NaN
13722,2024-02-02,united states,unemployment rate,unemployment rate,USURTOT,,3.7%,3.7%,NaN
13723,2024-02-05,australia,balance of trade,balance of trade,AUITGSB,,A$11.437B,A$ 7.9B,NaN
13724,2024-02-05,germany,balance of trade,balance of trade,GRTBALE,,€20.4B,€ 17B,NaN


In [13]:
calendar_data_df_jp = calendar_data_df[calendar_data_df.country == "united kingdom"].copy()

In [14]:
calendar_data_df_jp.head(10)

,date,country,category,event,symbol,actual,previous,forecast,data
642,2024-02-01,united kingdom,interest rate,boe interest rate decision,UKBRBASE,,5.25%,5.25%,NaN
689,2024-02-01,united kingdom,interest rate,boe interest rate decision,UKBRBASE,,5.25%,5.25%,NaN
697,2024-02-01,united kingdom,interest rate,boe interest rate decision,UKBRBASE,,5.25%,5.25%,NaN
763,2024-02-01,united kingdom,interest rate,boe interest rate decision,UKBRBASE,,5.25%,5.25%,NaN
791,2024-02-01,united kingdom,interest rate,boe interest rate decision,UKBRBASE,,5.25%,5.25%,NaN
822,2024-02-01,united kingdom,interest rate,boe interest rate decision,UKBRBASE,,5.25%,5.25%,NaN
856,2024-02-01,united kingdom,interest rate,boe interest rate decision,UKBRBASE,,5.25%,5.25%,NaN
887,2024-02-01,united kingdom,interest rate,boe interest rate decision,UKBRBASE,,5.25%,5.25%,NaN
947,2024-02-01,united kingdom,interest rate,boe interest rate decision,UKBRBASE,,5.25%,5.25%,NaN
975,2024-02-01,united kingdom,interest rate,boe interest rate decision,UKBRBASE,,5.25%,5.25%,NaN


In [17]:
calendar_data_df_jp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407 entries, 642 to 13719
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      407 non-null    object 
 1   country   407 non-null    object 
 2   category  407 non-null    object 
 3   event     407 non-null    object 
 4   symbol    407 non-null    object 
 5   actual    407 non-null    float64
 6   previous  407 non-null    float64
 7   forecast  407 non-null    float64
 8   data      0 non-null      object 
dtypes: float64(3), object(6)
memory usage: 31.8+ KB


In [16]:
for col in ['actual', 'previous', 'forecast']:
    for sy in ['£', '%', 'B', 'K']:
        calendar_data_df_jp[col] = calendar_data_df_jp[col].str.replace(sy, "")
    calendar_data_df_jp[col].replace('', 0, inplace=True)
    calendar_data_df_jp[col] = calendar_data_df_jp[col].astype(float)

In [18]:
date_match= {}
for orig in calendar_data_df_jp.date.unique():
    d = orig
    tries = 0
    while d not in candles_df.time.values:
        d = d + dt.timedelta(days=1)
        tries += 1
        if tries > 5:
            print("Failed")
            break
    date_match[orig] = d

Failed


In [19]:
date_match

{datetime.date(2024, 2, 1): datetime.date(2024, 2, 7)}

In [20]:
calendar_data_df_jp['orig_date'] = calendar_data_df_jp.date
calendar_data_df_jp.date = [date_match[x] for x in calendar_data_df_jp.date]  

In [21]:
calendar_data_df_jp['delta_prev'] = calendar_data_df_jp.actual - calendar_data_df_jp.previous
calendar_data_df_jp['delta_fc'] = calendar_data_df_jp.actual - calendar_data_df_jp.forecast

In [22]:
calendar_data_df_jp.head()

,date,country,category,event,symbol,actual,previous,forecast,data,orig_date,delta_prev,delta_fc
642,2024-02-07,united kingdom,interest rate,boe interest rate decision,UKBRBASE,0.0,5.25,5.25,NaN,2024-02-01,-5.25,-5.25
689,2024-02-07,united kingdom,interest rate,boe interest rate decision,UKBRBASE,0.0,5.25,5.25,NaN,2024-02-01,-5.25,-5.25
697,2024-02-07,united kingdom,interest rate,boe interest rate decision,UKBRBASE,0.0,5.25,5.25,NaN,2024-02-01,-5.25,-5.25
763,2024-02-07,united kingdom,interest rate,boe interest rate decision,UKBRBASE,0.0,5.25,5.25,NaN,2024-02-01,-5.25,-5.25
791,2024-02-07,united kingdom,interest rate,boe interest rate decision,UKBRBASE,0.0,5.25,5.25,NaN,2024-02-01,-5.25,-5.25


In [23]:
candles_df.head(2)

,volume,time,mid-o,mid-h,mid-l,mid-c,bid-o,bid-h,bid-l,bid-c,ask-o,ask-h,ask-l,ask-c,gain,pair
0,123541,2022-07-12,0.84427,0.84682,0.84032,0.84584,0.84386,0.84674,0.84026,0.84569,0.84468,0.84690,0.84039,0.84599,-0.185614,EUR_GBP
1,128494,2022-07-13,0.84580,0.84990,0.84406,0.84742,0.84527,0.84980,0.84397,0.84730,0.84632,0.85002,0.84414,0.84753,-0.191168,EUR_GBP


In [24]:
candles_an = candles_df[['time', 'pair', 'gain']].copy()

In [25]:
candles_an.head(2)

,time,pair,gain
0,2022-07-12,EUR_GBP,-0.185614
1,2022-07-13,EUR_GBP,-0.191168


In [26]:
merged = pd.merge(left=candles_an, right=calendar_data_df_jp, left_on='time', right_on='date')

In [27]:
common_dates = set(candles_an['time']).intersection(set(calendar_data_df_jp['date']))
print("Common Dates:", common_dates)

Common Dates: set()


In [28]:
print(merged.head())

Empty DataFrame
Columns: [time, pair, gain, date, country, category, event, symbol, actual, previous, forecast, data, orig_date, delta_prev, delta_fc]
Index: []
